In [352]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import morton
import math
from hilbertcurve.hilbertcurve import HilbertCurve

In [353]:
# Create 2D data
df = pd.DataFrame(np.random.randint(0,65,size=(5, 2)), columns=list('AB'))
df.describe()

,A,B
count,5.000000,5.000000
mean,26.200000,34.000000
std,20.315019,20.012496
min,0.000000,12.000000
25%,16.000000,18.000000
50%,29.000000,31.000000
75%,31.000000,52.000000
max,55.000000,57.000000


In [354]:
# Morton algorithm
m = morton.Morton(dimensions=2, bits=64)
def set_value(data,key1,key2):    
    return m.pack(int(data[key1]), int(data[key2]))



In [355]:
hilbert_curve = HilbertCurve(64, 2, n_procs=-1)
def set_value_Hcurve(data,key1,key2,key3):
    points = data[[key1,key2]].to_numpy()
    distances = hilbert_curve.distances_from_points(points)
    data[key3] = pd.DataFrame(distances )
    return data

In [360]:
# Offset experiment
offset_2 = 0
offset_3 = 256
df['A_offset_2']=df['A'].add(offset_2)
df['B_offset_2']=df['B'].add(offset_2)
df['A_offset_3']=df['A'].add(offset_3)
df['B_offset_3']=df['B'].add(offset_3)
df['morton_offset_2']= df.apply(set_value,args=('A_offset_2','B_offset_2'),axis=1)
df['morton_offset_3']= df.apply(set_value,args=('A_offset_3','B_offset_3'),axis=1)
df['offset_delta']=df['morton_offset_3']-df['morton_offset_2']
df['offset_delta']
df['offset_delta'].describe()
# plt.hist(df['offset_delta'],bins=10)


count         5.0
mean     196608.0
std           0.0
min      196608.0
25%      196608.0
50%      196608.0
75%      196608.0
max      196608.0
Name: offset_delta, dtype: float64

In [361]:
offset_2 = 0
offset_3 = 384 
df['A_offset_2']=df['A'].add(offset_2)
df['B_offset_2']=df['B'].add(offset_2)
df['A_offset_3']=df['A'].add(offset_3)
df['B_offset_3']=df['B'].add(offset_3)
df=set_value_Hcurve(df,'A_offset_2','B_offset_2','hcurve_offset_2')
df=set_value_Hcurve(df,'A_offset_3','B_offset_3','hcurve_offset_3')
df['offset_delta_hcurve']=df['hcurve_offset_3']-df['hcurve_offset_2']
df['offset_delta_hcurve']
df['offset_delta_hcurve'].describe()

# 3k*2^n where 2^n>max(df) for hilbert curve offset

count         5.0
mean     163840.0
std           0.0
min      163840.0
25%      163840.0
50%      163840.0
75%      163840.0
max      163840.0
Name: offset_delta_hcurve, dtype: float64

In [ ]:
df

In [ ]:
fig,ax=plt.subplots(1,2)
ax[0].eventplot(df['morton_offset_2'],orientation='vertical',linewidths=0.5)
ax[1].eventplot(df['morton_offset_3'],orientation='vertical',linewidths=0.5)
fig.show()

In [ ]:
# conclusion: for lowest 2^n>max(df), offset k*2^n will keep the shape of the CSP

In [ ]:
# Multiplier experiment
multi_2 = 1
multi_3 = 8
df['A_multi_2']=df['A'].multiply(multi_2)
df['B_multi_2']=df['B'].multiply(multi_2)
df['A_multi_3']=df['A'].multiply(multi_3)
df['B_multi_3']=df['B'].multiply(multi_3)
df['morton_multi_2']= df.apply(set_value,args=('A_multi_2','B_multi_2'),axis=1)
df['morton_multi_3']= df.apply(set_value,args=('A_multi_3','B_multi_3'),axis=1)
df['multi_delta']=df['morton_multi_3']/df['morton_multi_2']
df['multi_delta']
df['multi_delta'].describe()
# plt.hist(df['offset_delta'],bins=10)

In [ ]:
# Multiplier experiment h-curve
multi_2 = 1
multi_3 = 30
df['A_multi_2']=df['A'].multiply(multi_2)
df['B_multi_2']=df['B'].multiply(multi_2)
df['A_multi_3']=df['A'].multiply(multi_3)
df['B_multi_3']=df['B'].multiply(multi_3)
df=set_value_Hcurve(df,'A_multi_2','B_multi_2','hcurve_multi_2')
df=set_value_Hcurve(df,'A_multi_3','B_multi_3','hcurve_multi_3')
df['morton_multi_2']= df.apply(set_value,args=('A_multi_2','B_multi_2'),axis=1)
df['morton_multi_3']= df.apply(set_value,args=('A_multi_3','B_multi_3'),axis=1)
df['multi_delta_hcurve']=df['hcurve_multi_3']/df['hcurve_multi_2']
print(df)
df['multi_delta_hcurve'].describe()
# plt.hist(df['offset_delta'],bins=10)

In [ ]:
def get_std_with_different_multiplier(data,key1,key2,multiplier):
    col_name_1=key1+'_'+str(multiplier)
    col_name_2=key2+'_'+str(multiplier)
    data[col_name_1]=data[key1].multiply(multiplier)
    data[col_name_2]=data[key2].multiply(multiplier)
    data=set_value_Hcurve(data,col_name_1,col_name_2,'hcurve_multi')
    data=set_value_Hcurve(data,key1,key2,'origin')
    data['multi_delt_hcurve']=df['hcurve_multi']/df['origin']
    return data['multi_delt_hcurve'].std()


In [ ]:
std_list=[]
for i in range(2,100):
    std_list.append(get_std_with_different_multiplier(df,'A','B',i))
dff=pd.DataFrame(std_list,columns=['std'])
dff
# multiplier does not work with the h-curve

In [ ]:
dff.min()

In [ ]:
fig,ax=plt.subplots(1,2)
ax[0].eventplot(df['morton_multi_2'],orientation='vertical',linewidths=0.5)
ax[1].eventplot(df['morton_multi_3'],orientation='vertical',linewidths=0.5)
fig.show()

In [ ]:
# Conclusion: multipler should be 2^n

In [ ]:
# Does Multiply and offset order matter Experiment: Answer yes! the order matters


In [ ]:
fig,ax=plt.subplots(1,2)
ax[0].eventplot(df['morton_multi_2'],orientation='vertical',linewidths=0.5)
ax[1].eventplot(df['morton_multi_3'],orientation='vertical',linewidths=0.5)
fig.show()